In [2]:
import os
%pwd

'/home/towet/Desktop/Visions/tumors/Multimodal-Brain-Tumor-Segmentation/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/towet/Desktop/Visions/tumors/Multimodal-Brain-Tumor-Segmentation'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    train_URL: str
    val_URL: str
    unzip_dir: Path

In [6]:
from tumorsegmentation.constants import *
from tumorsegmentation.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            train_URL=config.train_URL,
            val_URL=config.val_URL,
            
        
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [8]:
import os
import urllib.request as request
import zipfile
from tumorsegmentation import logger
from tumorsegmentation.utils.common import get_size
import requests

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_train_data(self):
        self.local_filename = self.config.train_URL.split('/')[-1]
        # NOTE the stream=True parameter below
        with requests.get(self.config.train_URL, stream=True) as r:
            r.raise_for_status()
            with open(self.local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        return self.local_filename
    
    def download_val_data(self):
        self.local_filename = self.config.val_URL.split('/')[-1]
        # NOTE the stream=True parameter below
        with requests.get(self.config.train_URL, stream=True) as r:
            r.raise_for_status()
            with open(self.local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        return self.local_filename
    
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.download_train_data(), 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        with zipfile.ZipFile(self.download_val_data(), 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-24 18:07:49,366: INFO: common: yaml file: config/config.yaml loaded successfully]


[2023-11-24 18:07:49,435: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-24 18:07:49,436: INFO: common: created directory at: artifacts]
[2023-11-24 18:07:49,439: INFO: common: created directory at: artifacts/data_ingestion]


KeyboardInterrupt: 